# Contextual data

Once you've identified areas of interest where multiple datasets intersect, you can pull additional data to provide further context. For example:

1. landcover 
2. global elevation data 

In [1]:
import coincident
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np

%matplotlib inline

## Identify a primary dataset

Start by loading a full resolution polygon of a 3DEP LiDAR workunit which has a known start_datetime and end_datatime:

In [ ]:
workunit = "CO_WestCentral_2019"
df_wesm = coincident.search.wesm.read_wesm_csv()
gf_lidar = coincident.search.wesm.load_by_fid(
    df_wesm[df_wesm.workunit == workunit].index
)

gf_lidar

In [6]:
search_aoi = gf_lidar.simplify(0.01)

## Search for coincident contextual data

Coincident provides two convenience functions to load datasets resulting from a given search. 

In [7]:
gf_wc = coincident.search.search(
    dataset="worldcover",
    intersects=search_aoi,
    # worldcover is just 2020 an 2021, so pick one
    datetime=["2020"],
)  # Asset of interest = 'map'

In [ ]:
# STAC metadata always has a "stac_version" column
gf_wc.iloc[0].stac_version

In [ ]:
gf_cop30 = coincident.search.search(
    dataset="cop30",
    intersects=search_aoi,
)  # Asset of interest = 'data'
gf_cop30.iloc[0].stac_version

### STAC search results to datacube

If the results have [STAC-formatted metadata](https://stacspec.org/en). We can take advantage of the excellent [odc.stac](https://odc-stac.readthedocs.io/) tool to load datacubes. Please refer to odc.stac documentation for all the configuration options (e.g. setting resolution our output CRS, etc.)

By default this uses the [dask](https://www.dask.org/) parallel computing library to intelligently load only metadata initially and defer reading values until computations or visualizations are performed.

#### Copernicus DEM

In [10]:
ds = coincident.io.xarray.to_dataset(
    gf_cop30,
    aoi=search_aoi,
    # chunks=dict(x=2048, y=2048), # manual chunks
    resolution=0.00081,  # ~90m
    mask=True,
)

In [ ]:
# By default, these are dask arrays
ds

In [15]:
# You might want to rename the data variable
ds = ds.rename(data="elevation")

In [ ]:
# The total size of this dataset is only 14MB, so for faster computations, load it into memory
print(ds.nbytes / 1e6)
ds = ds.compute()

In [ ]:
ds.elevation.isel(time=0).plot.imshow();

#### ESA Worldcover

In [ ]:
# Same with LandCover
dswc = coincident.io.xarray.to_dataset(
    gf_wc,
    bands=["map"],
    aoi=search_aoi,
    mask=True,
    # resolution=0.00027, #~30m
    resolution=0.00081,  # ~90m
)
dswc

In [21]:
dswc = dswc.rename(map="landcover")

In [22]:
dswc = dswc.compute()

In [ ]:
# For landcover there is a convenicence function for a nice categorical colormap
ax = coincident.plot.plot_esa_worldcover(dswc)
ax.set_title("ESA WorldCover");

## Load gridded elevation in a consistent CRS

`coincident` also has a convenience function for loading gridded elevation datasets in a consistent CRS. In order to facilitate comparison with modern altimetry datasets (ICESat-2, GEDI), we convert elevation data on-the-fly to [EPSG:7912](https://spatialreference.org/ref/epsg/7912/). 3D Coordinate Reference Frames are a complex topic, so please see this resource for more detail on how these conversions are done: https://uw-cryo.github.io/3D_CRS_Transformation_Resources/ 

```{note}
currently data is retrieved from [OpenTopography](https://opentopography.org/) (hosted in AWS us-west-2) although many mirrors of global DEM datasets exist.
```

```{warning}
 loading data with odc.stac, the native resolution of the grid is used, and data is immediately read into memory so this method is better suited to small AOIs.
```

In [ ]:
# Start with a small AOI:
from shapely.geometry import box
import geopandas as gpd

aoi = gpd.GeoDataFrame(
    geometry=[box(-106.812163, 38.40825, -106.396812, 39.049796)], crs="EPSG:4326"
)

In [ ]:
da = coincident.io.xarray.load_dem_7912("cop30", aoi=aoi)
da